# Seldon V2 Pipeline async drift detection example


 * Build if needed and place `seldon` binary in your path
    * run `make build-seldon` from operator folder and add bin folder to PATH
 * Run Seldon V2 `make deploy-local` from top level folder
 * Install `kafkacat` for debugging kafka topics & messages

In [ ]:
!which seldon

In [ ]:
!kafkacat -b localhost:9092 -L | grep -i seldon -A1

### Train the drift detector for an iris model using alibi detect

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()
X, y = iris.data, iris.target

In [ ]:
from alibi_detect.cd import TabularDrift
cd_tabular = TabularDrift(X, p_val=.05)

In [ ]:
from alibi_detect.utils.saving import save_detector
filepath = "alibi-detector-artifacts"
save_detector(cd_tabular, filepath)

In [ ]:
# !gsutil -m cp ./alibi-detector-artifacts/ gs://seldon-models/mlserver/alibi-detect/iris-tabular-drift/

## Model Batched Chaining based on batch size

In [ ]:
!cat ../../samples/models/sklearn-iris-gs.yaml
!cat ../../samples/models/iris-drift-detect.yaml

In [ ]:
!seldon model load -f ../../samples/models/sklearn-iris-gs.yaml 
!seldon model load -f ../../samples/models/iris-drift-detect.yaml

In [ ]:
!seldon model status --model-name iris -w ModelAvailable | jq .
!seldon model status --model-name iris-drift-detect -w ModelAvailable | jq .

In [ ]:
!cat ../../samples/pipelines/iris-async-drift.yaml

In [ ]:
!seldon pipeline load -f ../../samples/pipelines/iris-async-drift.yaml

In [ ]:
!seldon pipeline status -p iris-async-drift -w PipelineReady| jq .

In [ ]:
!seldon pipeline infer -p iris-async-drift --inference-mode grpc \
    '{"model_name":"iris","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4]},"datatype":"INT32","shape":[1,4]}]}' | jq .

In [ ]:
!kafkacat -b localhost:9092 -t seldon.default.model.iris-drift-detect.outputs -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

In [ ]:
!kafkacat -b localhost:9092 -t seldon.default.model.iris-drift-detect.inputs -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

In [ ]:
!seldon pipeline infer -p iris-async-drift --inference-mode grpc \
    '{"model_name":"iris","inputs":[{"name":"INPUT0","contents":{"int_contents":[1,2,3,4]},"datatype":"INT32","shape":[1,4]}]}' | jq .

In [ ]:
!kafkacat -b localhost:9092 -L | grep -i seldon -A1

In [ ]:
!kafkacat -b localhost:9092 -t seldon.default.pipeline.iris-async-drift.inputs -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

In [ ]:
!kafkacat -b localhost:9092 -t seldon.default.model.iris.inputs -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

In [ ]:
!kafkacat -b localhost:9092 -t seldon.default.model.iris.outputs -C -o beginning -e -f 'Offset %o\tlength %S\tcontents %s\n' -s value=i

### Cleanup

In [ ]:
!seldon pipeline unload -p iris-async-drift

In [ ]:
!seldon model unload --model-name iris
!seldon model unload --model-name iris-drift-detect